In [61]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import os
import math
import shutil
import glob


In [62]:
# count the number of images in the classes here, 0= tumour; 1= no tumour
ROOT_DIR = r'C:\Users\lenovo\Desktop\Code\Mini Project\Tumour Detection\Training Data'
number_of_img ={}

for dir in os.listdir(ROOT_DIR):
    number_of_img[dir] = len(os.listdir(os.path.join(ROOT_DIR, dir)))
    
    #print(number_of_img.items())

We will **split data** in such a manner that:

**70%** of the data will be used for **training**. 

**15%** of the data will be used for **validation**. 

**15%** of the data will be used for **testing**.

In [63]:
# we will create a training folder

def dataFolderCreator(path, split):

    if not os.path.exists('./'+path):
        os.makedirs('./'+path)

        for dir in os.listdir(ROOT_DIR):
            os.makedirs("./"+path+"/"+dir)
            for img in np.random.choice(a= os.listdir(os.path.join(ROOT_DIR, dir)), size=(math.floor(split * number_of_img[dir])-5), replace=False):
        
                o= os.path.join(ROOT_DIR, dir, img) #path
                D= os.path.join("./"+path,dir)
                shutil.copy(o, D) #copy the file to the new folder
                
                # os.remove(o) -> (uncomment this line to remove the original file after copying) 

    else:
        print(f"{path} folder exists")

In [64]:
dataFolderCreator('TrainingData', 0.7)
dataFolderCreator('ValidationData', 0.15)
dataFolderCreator('TestingData', 0.15)



TrainingData folder exists
ValidationData folder exists
TestingData folder exists


## *Model Building*

In [65]:
import keras
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# CNN Model

model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=(224, 224, 3), padding='same'))
model.add(Conv2D(filters=36, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(units=64, activation='relu'))

model.add(Dropout(rate=0.25))
model.add(Dense(units=1, activation='sigmoid'))

model.summary()


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_16 (Conv2D)              │ (None, 224, 224, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 222, 222, 36)   │         5,220 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 111, 111, 36)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 109, 109, 64)   │        20,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │     5,537,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,638,245 (21.51 MB)

 Trainable params: 5,638,245 (21.51 MB)

 Non-trainable params: 0 (0.00 B)

In [66]:
# Model Compilation

model.compile(optimizer='adam', loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Preparing our Data using DataGenerator

In [67]:
#Training Data
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
def preProcessingImage1(path):
    ''' Input: Path of the image
    Output: Preprocessed image
    '''
    
    image_data = ImageDataGenerator(zoom_range=0.2, shear_range=0.2, preprocessing_function=preprocess_input, horizontal_flip=True) #data augmentation
    image = image_data.flow_from_directory(directory=path, target_size=(224, 224), batch_size=32, class_mode='binary')
    
    return image

In [68]:
training_data = preProcessingImage1(r'C:\Users\lenovo\Desktop\Code\Mini Project\Tumour Detection\Training Data')

Found 2065 images belonging to 2 classes.


In [69]:
# Validation Data
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
def preProcessingImage2(path):
    ''' Input: Path of the image
    Output: Preprocessed image
    '''
    
    image_data = ImageDataGenerator(preprocessing_function=preprocess_input)
    image = image_data.flow_from_directory(directory=path, target_size=(224, 224), batch_size=32, class_mode='binary')
    
    return image

In [70]:
test_data = preProcessingImage2(r'C:\Users\lenovo\Desktop\Code\Mini Project\Tumour Detection\TestingData')

val_data = preProcessingImage2(r'C:\Users\lenovo\Desktop\Code\Mini Project\Tumour Detection\ValidationData')

Found 299 images belonging to 2 classes.
Found 299 images belonging to 2 classes.


In [71]:
# Early Stopping and Model Checkpoint

from keras.callbacks import EarlyStopping, ModelCheckpoint

# Early Stopping
early_stopping = EarlyStopping(monitor='val_accuracy',min_delta= 0.01, patience=3, verbose=1, mode='auto')

# Model Checkpoint
model_checkpoint = ModelCheckpoint(filepath='./best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1, mode='auto')

callBacks = [early_stopping, model_checkpoint]

## *Model Training*

In [72]:
# Model Training

import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Flatten, Dense
from keras.models import Model, load_model
from keras.applications.mobilenet import MobileNet, preprocess_input


base_model = MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

for layer in base_model.layers:
    layer.trainable = False
    



In [2]:
# base_model.summary()
import keras

X = Flatten()(base_model.output)
X = Dense(units=1, activation='sigmoid')(X)

model = Model(base_model.input, X)
model.compile(loss=keras.losses.binary_crossentropy, optimizer='rmsprop', metrics=['accuracy'])

NameError: name 'Flatten' is not defined

In [74]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

# model checkpoint
MC = ModelCheckpoint(filepath='./best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

# early stopping
ES = EarlyStopping(monitor='val_accuracy', min_delta=0.01, patience=3, verbose=1)

CB = [MC, ES]


In [1]:
History = model.fit(training_data, steps_per_epoch=8, epochs=30, validation_data=val_data, validation_steps=16, callbacks=CB)

NameError: name 'model' is not defined

In [76]:
# load the best fit model

model = load_model('./best_model.h5')

In [6]:
# Model Graphical Representation

h= History.history
h.keys()

# Plotting Accuracy

plt.plot(h['accuracy'], label='Training Accuracy')
plt.plot(h['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
# plt.show() #-> remove '#' to see the accuracy graph

# Plotting Loss
plt.plot(h['loss'], label='Training Loss')
plt.plot(h['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
# plt.show() #-> remove '#' to see the loss graph

NameError: name 'History' is not defined

In [78]:
# Loading the best model

from keras.models import load_model

model = load_model(r'C:\Users\lenovo\Desktop\Code\Mini Project\Tumour Detection\best_model.h5')

## *Model Accuracy*

In [80]:
acc = model.evaluate(test_data)

print(f"Accuracy of our model is: ", (acc[1] * 100), "%")

10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 578ms/step - accuracy: 0.9078 - loss: 0.5979
Accuracy of our model is:  91.63879752159119 %


In [ ]:
# to check if a random image has tumour or not

from keras.preprocessing import image

path=r'C:\Users\lenovo\Desktop\Code\Mini Project\Tumour Detection\Training Data\no\aug_10 no_0_2521.jpg'

img= load_img(path, target_size=(224,224))
input_img = image.img_to_array(img)/255   # we divide by 255 to normalize the image
input_img = preprocess_input(input_img)
plt.imshow(img)
plt.show()

input_img = np.array(input_img)
input_img.shape

prediction = np.argmax(model.predict(input_img))

if prediction == 0:
    print("The image has a tumour")
else:
    print("The image does not have a tumour")

# to check the class indices of the training data
training_data.class_indices

NameError: name 'plt' is not defined